In [1]:
using RigidBodyDynamics
using StaticArrays
using RigidBodyDynamics.TreeDataStructure
import RigidBodyDynamics: VectorSegment
import RigidBodyDynamics: num_positions, num_velocities
using BenchmarkTools

In [2]:
const ScalarType = Float64
# const ScalarType = Float32

function create_floating_atlas()
    atlasUrdfUrl = "https://raw.githubusercontent.com/RobotLocomotion/drake/6e3ca768cbaabf15d0f2bed0fb5bd703fa022aa5/drake/examples/Atlas/urdf/atlas_minimal_contact.urdf"
    atlasUrdf = RigidBodyDynamics.cached_download(atlasUrdfUrl, "atlas.urdf")
    atlas = parse_urdf(ScalarType, atlasUrdf)
#     for child in children(root_vertex(atlas))
#         joint = edge_to_parent_data(child)
#         change_joint_type!(atlas, joint, QuaternionFloating{ScalarType}())
#     end
    atlas
end
const mechanism = create_floating_atlas()
remove_fixed_joints!(mechanism);

In [3]:
const mechanism = rand_tree_mechanism(Float64, [QuaternionFloating{Float64}; [Revolute{Float64} for i = 1 : 15]; [Prismatic{Float64} for i = 1 : 15]]...);

In [4]:
include(joinpath(Pkg.dir("RigidBodyDynamics"), "src", "codegen.jl"))
using RBDCodeGen

In [5]:
const state = RBDCodeGen.MechanismState(Float64, mechanism);

In [6]:
# @benchmark t1 * t2 setup = (element = state.elements[4]; t1 = element.beforeJointToParent; j = element.joint; q = element.q; t2 = joint_transform(j, q))
# @benchmark X1 * X2 setup = (X1 = rand(SMatrix{4, 4}); X2 = rand(SMatrix{4, 4}))
# @benchmark (R1 * R2, R1 * p2 + p1) setup = (R1 = rand(SMatrix{3, 3}); R2 = rand(SMatrix{3, 3}); p1 = rand(SVector{3}); p2 = rand(SVector{3}))

In [7]:
nv = num_velocities(mechanism)
nq = num_positions(mechanism)
const M = Symmetric(Matrix{Float64}(nv, nv), :L);

In [8]:
@benchmark update_joint_transforms($state) setup = (rand!(state.q); rand!(state.v))

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.625 μs (0.00% GC)
  median time:      2.757 μs (0.00% GC)
  mean time:        3.040 μs (0.00% GC)
  maximum time:     9.794 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     9
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [9]:
@benchmark update_crb_inertias($state) setup = (rand!(state.q); rand!(state.v))

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.120 μs (0.00% GC)
  median time:      2.207 μs (0.00% GC)
  mean time:        2.414 μs (0.00% GC)
  maximum time:     9.117 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [10]:
@time mass_matrix!(M, state)

  1.094095 seconds (792.41 k allocations: 32.955 MB, 0.39% gc time)


In [11]:
@benchmark mass_matrix!($M, $state) setup = (rand!(state.q); rand!(state.v))

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     13.373 μs (0.00% GC)
  median time:      14.137 μs (0.00% GC)
  mean time:        15.089 μs (0.00% GC)
  maximum time:     103.603 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [12]:
# test(n) = for i = 1 : n mass_matrix!(M, state) end
# # test(n) = for i = 1 : n update_joint_transforms(state) end
# using ProfileView
# test(1)
# Profile.clear()
# @profile test(100000)
# ProfileView.view()

In [13]:
using ForwardDiff

In [14]:
const chunk_size = 1

1

In [15]:
nv = num_velocities(mechanism)
const state_autodiff = RBDCodeGen.MechanismState(ForwardDiff.Dual{chunk_size, Float64}, mechanism);
const M_autodiff = Symmetric(Matrix{ForwardDiff.Dual{chunk_size, Float64}}(nv, nv), :U);

In [16]:
function f!(out, q)
    copy!(state_autodiff.q, q)
    mass_matrix!(M_autodiff, state_autodiff)
    @simd for i in eachindex(out)
        out[i] = M_autodiff[i]
    end
end

f! (generic function with 1 method)

In [17]:
const out = Matrix{Float64}(nv * nv, nq)
const y = Vector{Float64}(nv * nv)
@time ForwardDiff.jacobian!(out, f!, y, zeros(nv), ForwardDiff.JacobianConfig{chunk_size}(y, zeros(nq)));

  9.457854 seconds (10.18 M allocations: 234.928 MB, 0.57% gc time)


In [18]:
@benchmark ForwardDiff.jacobian!($out, $f!, $y, q, config) setup = (q = rand(num_positions(mechanism)); config = ForwardDiff.JacobianConfig{chunk_size}(y, q))

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.728 ms (0.00% GC)
  median time:      1.898 ms (0.00% GC)
  mean time:        1.980 ms (0.00% GC)
  maximum time:     3.154 ms (0.00% GC)
  --------------
  samples:          2497
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [19]:
# const q = rand(num_velocities(mechanism));
# const config = ForwardDiff.JacobianConfig{chunk_size}(y, q)
# test2(n) = for i = 1 : n ForwardDiff.jacobian!(out, f!, y, q, config) end

In [20]:
# using ProfileView
# test2(1)
# Profile.clear()
# @profile test2(10000)
# ProfileView.view()